# Agentic RAG with Llama Stack

This notebook highlights how to integrate **Docling MCP** tools in the Agentic RAG available in Llama Stack.

We will use the Llama Stack framework. To get an introduction to Llama Stack capabilities, including its current builtin tools, you can refer to the
[Llama Stack Demos on OpenDataHub](https://github.com/opendatahub-io/llama-stack-demos) repository.

This example will use the inline Milvus component available in the Llama Stack distributions.

### Tools:

We will use tools internal to Llama Stack and from the Docling MCP server that allow executing tasks such as:
- [`mcp:docling`] converting a PDF file from a local or remote location into a unified document representation [DoclingDocument](https://docling-project.github.io/docling/concepts/docling_document/).
- [`mcp:docling`] chunk and ingest the document in the Llama Stack vectordb.
- [`builtin::rag/knowledge_search`] search in the document using agentic rag techniques.

## Pre-Requisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [README.md](./README.md) file to set up the following resources:
  - Inference model with Ollama
  - Llama Stack server with the Ollama template [distribution-starter](https://hub.docker.com/r/llamastack/distribution-starter)
  - Docling MCP server 

You may want to create a virtual environment to run this notebook, for instance, with [uv](https://docs.astral.sh/uv/). Ensure to install the llama-stack optional dependencies, as well as the examples group dependencies:

```bash
uv venv
source .venv/bin/activate
uv sync --extra llama-stack --group examples
```


## Setting Up this Notebook

Rename or copy the [`.env.example`](./.env.examples) file to create a new file called `.env`. Most environmental variables are already set up with default values to run this notebook and they are aligned to the set up of the pre-requisites, like the Llama Stack server and the Docling MCP endpoints.

```bash
cp .env.example .env
```

### Environment variables required for this notebook

- `BASE_URL`: the URL of the remote Llama Stack server. Defaults to `http://localhost:8321`.
- `INFERENCE_MODEL`: the generative AI model id. Defaults to the Meta Llama 3.2 model (`meta-llama/Llama-3.2-3B-Instruct`).
- `TEMPERATURE` (optional): the temperature to use during inference. Defaults to 0.0.
- `TOP_P` (optional): the top_p parameter to use during inference. Defaults to 0.95.
- `MAX_TOKENS` (optional): the maximum number of tokens that can be generated in the completion. Defaults to 4096.
- `STREAM` (optional): set this to True to stream the output of the model/agent and False otherwise. Defaults to True.
- `USE_PROMPT_CHAINING`: dictates if the prompt should be formatted as a few separate prompts to isolate each step or in a single turn.
- `DOCLING_MCP_URL`: the URL for the Docling MCP server. If the client does not find the tool registered to the llama-stack instance, it will use this URL to register it.

## Necessary Imports

In [3]:
import logging
import uuid

from llama_stack_client import Agent
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from pydantic import NonNegativeFloat, AnyHttpUrl
from pydantic_settings import BaseSettings, SettingsConfigDict
from rich.pretty import pprint

from src.utils import step_printer, user_printer

# set the logger
logger = logging.getLogger(__name__)
if not logger.hasHandlers():
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter("%(message)s")
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)


# access the environment variables
class Settings(BaseSettings):
    base_url: AnyHttpUrl = "http://localhost:8321"
    inference_model: str = "ollama/llama3.2:3b-instruct-fp16"
    max_tokens: int = 4096
    temperature: NonNegativeFloat = 0.0
    top_p: float = 0.95
    stream: bool = False
    use_prompt_chaining: bool = True

    docling_mcp_url: AnyHttpUrl = "http://host.containers.internal:8000/mcp"

    vdb_provider: str = "milvus"
    vdb_embedding: str = "all-MiniLM-L6-v2"
    vdb_embedding_dimension: int = 384
    # vdb_embedding_window: int = 256

    model_config = SettingsConfigDict(
        env_file=".env", env_file_encoding="utf-8", extra="ignore"
    )


settings = Settings()
pprint(settings)

Settings(
│   base_url=AnyHttpUrl('http://localhost:8321/'),
│   inference_model='ollama/llama3.2:3b-instruct-fp16',
│   max_tokens=4096,
│   temperature=0.0,
│   top_p=0.95,
│   stream=False,
│   use_prompt_chaining=True,
│   docling_mcp_url=AnyHttpUrl('http://host.containers.internal:8000/mcp'),
│   vdb_provider='milvus',
│   vdb_embedding='all-MiniLM-L6-v2',
│   vdb_embedding_dimension=384
)

## Setting Up the Server Connection

Establish the connection to your Llama Stack server.

In [4]:
client = LlamaStackClient(base_url=str(settings.base_url))
print(f"Connected to Llama Stack server @ {client.base_url}")

Connected to Llama Stack server @ http://localhost:8321/


## Initializing the Inference Parameters

Fetch the inference-related parameters from the corresponding environment variables and convert them to the format Llama Stack expects.

In [5]:
if settings.temperature > 0.0:
    strategy = {
        "type": "top_p",
        "temperature": settings.temperature,
        "top_p": settings.top_p,
    }
else:
    strategy = {"type": "greedy"}

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": settings.max_tokens,
}

print(
    f"Inference Parameters:\n\tSampling Parameters: {sampling_params}\n\tstream: {settings.stream}"
)

Inference Parameters:
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## Validate that the Docling MCP tools are available in the Llama Stack instance

When an instance of Llama Stack is redeployed, it may be the case that the tools will need to be re-registered. Also if a tool is already registered with a Llama Stack instance, trying to register another one with the same `toolgroup_id` will throw you an error.

For this reason, it is recommended to validate your tools and toolgroups. The following code will check that `mcp::docling` tools are correctly registered, and if not it will attempt to register them using their specific endpoints.

In [6]:
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
if "mcp::docling" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::docling",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri": str(settings.docling_mcp_url)},
    )

registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
print(
    f"Your Llama Stack server is registered with the following tool groups @ {set(registered_toolgroups)} \n"
)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 200 OK"


Your Llama Stack server is registered with the following tool groups @ {'builtin::websearch', 'mcp::docling', 'builtin::rag'} 



## Defining our Agent - Prompt Chaining

We define an agent provided with the **Docling MCP** tools together with the built-in knowledge_search. The agent should be able to accomplish the following tasks in a multi-step, multi-tool approach:

1. Converting a PDF file into the `DoclingDocument` format.
2. Ingest the results in the vector db.
3. Search in the vector db using an agentic/multi-step approach.

In [7]:
model_prompt = """You are a helpful assistant. You have access to a number of tools.
Whenever a tool is called, be sure to return the Response in a friendly and helpful tone.
"""

In [22]:
# define the name of the vectordb collection to use
vector_db_id = f"test_vector_db_{uuid.uuid4()}"

# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=settings.vdb_embedding,
    embedding_dimension=settings.vdb_embedding_dimension,
    provider_id=settings.vdb_provider,
)


# Create simple agent with tools
agent = Agent(
    client=client,
    model=settings.inference_model,  # replace this with model_id to get the value of INFERENCE_MODEL_ID environment variable
    instructions=model_prompt,  # update system prompt based on the model you are using
    tools=[
        dict(
            name="mcp::docling/convert_pdf_document_into_docling_document",
            args={},
        ),
        dict(
            name="mcp::docling/insert_document_to_vectordb",
            args={
                "vector_db_id": vector_db_id,
            },
        ),
        dict(
            name="builtin::rag/knowledge_search",
            args={
                "vector_db_ids": [
                    vector_db_id
                ],  # list of IDs of document collections to consider during retrieval
            },
        ),
    ],
    tool_config={"tool_choice": "auto"},
    sampling_params=sampling_params,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-dbs "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling%2Fconvert_pdf_document_into_docling_document "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling%2Finsert_document_to_vectordb "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Arag%2Fknowledge_search "HTTP/1.1 200 OK"


In [23]:
user_prompts = [
    "Convert the PDF document on https://arxiv.org/pdf/2206.01062 to DoclingDocument.",
    "Ingest the document.",
    "Answer with the document knowledge in the vectordb: How many pages were manually annotated?",
]
session_id = agent.create_session(f"docling-session_{uuid.uuid4()}")

for i, prompt in enumerate(user_prompts):
    user_printer(prompt)
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=settings.stream,
    )
    if settings.stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(
            response.steps
        )  # print the steps of an agent's response in a formatted way.

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bb6c105b-8528-4c6a-930a-9c90f36da2b7/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bb6c105b-8528-4c6a-930a-9c90f36da2b7/session/263c4dbf-fe54-4afd-8e2e-ae29cd2147a5/turn "HTTP/1.1 200 OK"


👤 User Query:
Convert the PDF document on https://arxiv.org/pdf/2206.01062 to DoclingDocument.

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: convert_pdf_document_into_docling_document, Arguments: {'source': 'https://arxiv.org/pdf/2206.01062'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='{\n  "success": true,\n  "document_key": "868f49ae1f0e66e82238a8aea43fd30b"\n}',
│   │   type='text'
│   )
]

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bb6c105b-8528-4c6a-930a-9c90f36da2b7/session/263c4dbf-fe54-4afd-8e2e-ae29cd2147a5/turn "HTTP/1.1 200 OK"



---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The PDF document has been successfully converted to a Docling document and stored in the local cache. The unique key for this document is '868f49ae1f0e66e82238a8aea43fd30b'. You can use this key to insert the document into a vector database or perform knowledge searches on it.

========== Query processing completed ========== 

👤 User Query:
Ingest the document.

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: insert_document_to_vectordb, Arguments: {'document_key': '868f49ae1f0e66e82238a8aea43fd30b', 'vector_db_id': 'my_vector_db'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='{\n  "vector_db_id": "test_vector_db_2d8d7eb5-d0c6-4970-adab-4e72d6d980f7"\n}',
│   │   type='text'
│   )
]

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/bb6c105b-8528-4c6a-930a-9c90f36da2b7/session/263c4dbf-fe54-4afd-8e2e-ae29cd2147a5/turn "HTTP/1.1 200 OK"



---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The document has been successfully ingested into the vector database. The unique identifier for this vector database is 'test_vector_db_2d8d7eb5-d0c6-4970-adab-4e72d6d980f7'. You can now use this vector database to perform knowledge searches on the converted PDF document.

========== Query processing completed ========== 

👤 User Query:
Answer with the document knowledge in the vectordb: How many pages were manually annotated?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: knowledge_search, Arguments: {'query': 'number of pages manually annotated in 2206.01062'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text="Result 1\nContent: 3 THE DOCLAYNET DATASET\nDocLayNet contains 80863 PDF pages. Among these, 7059 carry two instances of human annotations, and 1591 carry three. This amounts to 91104 total annotation instances. The annotations provide layout information in the shape of labeled, rectangular boundingboxes. We define 11 distinct labels for layout features, namely Caption , Footnote , Formula , List-item , Page-footer , Page-header , Picture , Section-header , Table , Text , and Title . Our reasoning for picking this particular label set is detailed in Section 4.\nIn addition to open intellectual property constraints for the source documents, we required that the documents in DocLayNet adhere to a few conditions. Firstly, we kept scanned documents\nFigure 2: Distribution of DocLayNet pages across document categories.\nMetadata: {'chunk_id': '7156212269791437020-00012', 'document_id': '7156212269791437020', 'source': None, 'doc_items': ['#/texts/339', '#/texts/340', '#/texts/343']}\n",
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text="Result 2\nContent: 4 ANNOTATION CAMPAIGN\nwere carried out over a timeframe of 12 weeks, after which 8 of the 40 initially allocated annotators did not pass the bar.\nPhase 4: Production annotation. The previously selected 80K pages were annotated with the defined 11 class labels by 32 annotators. This production phase took around three months to complete. All annotations were created online through CCS, which visualises the programmatic PDF text-cells as an overlay on the page. The page annotation are obtained by drawing rectangular bounding-boxes, as shown in Figure 3. With regard to the annotation practices, we implemented a few constraints and capabilities on the tooling level. First, we only allow non-overlapping, vertically oriented, rectangular boxes. For the large majority of documents, this constraint was sufficient and it speeds up the annotation considerably in comparison with arbitrary segmentation shapes. Second, annotator staff were not able to see each other's annotations. This was enforced by design to avoid any bias in the annotation, which could skew the numbers of the inter-annotator agreement (see Table 1). We wanted\nMetadata: {'chunk_id': '7156212269791437020-00039', 'document_id': '7156212269791437020', 'source': None, 'doc_items': ['#/texts/422', '#/texts/423']}\n",
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text="Result 3\nContent: 4 ANNOTATION CAMPAIGN\nThe annotation campaign was carried out in four phases. In phase one, we identified and prepared the data sources for annotation. In phase two, we determined the class labels and how annotations should be done on the documents in order to obtain maximum consistency. The latter was guided by a detailed requirement analysis and exhaustive experiments. In phase three, we trained the annotation staff and performed exams for quality assurance. In phase four,\n\nTable 1: DocLayNet dataset overview. Along with the frequency of each class label, we present the relative occurrence (as % of row 'Total') in the train, test and validation sets. The inter-annotator agreement is computed as the mAP@0.5-0.95 metric between pairwise annotations from the triple-annotated pages, from which we obtain accuracy ranges.\nMetadata: {'chunk_id': '7156212269791437020-00019', 'document_id': '7156212269791437020', 'source': None, 'doc_items': ['#/texts/365', '#/texts/367', '#/tables/0']}\n",
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text="Result 4\nContent: 4 ANNOTATION CAMPAIGN\nTotal.Train = 5.10. Page-header, % of Total.Test = 6.70. Page-header, % of Total.Val = 5.06. Page-header, triple inter-annotator mAP @0.5-0.95 (%).All = 85-89. Page-header, triple inter-annotator mAP @0.5-0.95 (%).Fin = 66-76. Page-header, triple inter-annotat


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
According to the search results, there are a total of 5 pages that have been manually annotated in the document 2206.01062.

========== Query processing completed ========== 



## Defining our Agent - ReAct

We can also define our agent to be more autonomous and perform the same task with a single prompt instead of a chain. To do this, we leverage Llama Stack's **ReAct agent**, which has the ability to loop through the *Reason then Act* iterations, thinking through the problem and then using tools until it determines that it's task has been completed successfully.  

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In this example, we will leverage the generation tools of **Docling MCP** to:
- create a new `DoclingDocument`
- add a title and paragraphs
- export the resulting document into a file in markdown format

The purpose of the code below is just to show the instructions sent to the LLM to complete the task. They incorporate the description of the Docling MCP tools.

In [8]:
# ReAct instructions
from llama_stack_client.lib.agents.agent import AgentUtils
from llama_stack_client.lib.agents.react.agent import get_default_react_instructions

client_tools = AgentUtils.get_client_tools(["mcp::docling"])
instructions = get_default_react_instructions(client, ["mcp::docling"], client_tools)
print(instructions)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 200 OK"



You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to the following tools: is_document_in_local_cache, convert_pdf_document_into_docling_document, create_new_docling_document, export_docling_document_to_markdown, save_docling_document, add_title_to_docling_document, add_section_heading_to_docling_document, add_paragraph_to_docling_document, open_list_in_docling_document, close_list_in_docling_document, add_list_items_to_list_in_docling_document, add_table_in_html_format_to_docling_document

You must always respond in the following JSON format:
{
    "thought": $THOUGHT_PROCESS,
    "action": {
        "tool_name": $TOOL_NAME,
        "tool_params": $TOOL_PARAMS
    },
    "answer": $ANSWER
}

Specifically, this json should have a `thought` key, a `action` key and an `answer` key.

The `action` key should specify the $TOOL_NAME the name of the tool to use and the `tool_params` key 

In [12]:
agent = ReActAgent(
    client=client,
    model=settings.inference_model,
    tools=["mcp::docling"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

user_prompts = [
    """I want you to create a new Docling document on Quantum Computing.
    To do this, you will create a document by invoking `create_new_docling_document` with Quantum Computing as prompt.
    Add a title and 3 paragraphs describing the topic, one by one, e.g. using the add_x_to_docling_document tools.
    At the end of the writing, you must save the document by calling `save_docling_document` and return me the filepath of
    the saved document in markdown format provided by the last tool.
    """
]

session_id = agent.create_session(f"docling-session_{uuid.uuid4()}")
for prompt in user_prompts:
    print("\n" + "=" * 50)
    user_printer(prompt)
    print("=" * 50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=settings.stream,
    )
    if settings.stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(
            response.steps
        )  # print the steps of an agent's response in a formatted way.

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/00b00d76-1313-46cd-af96-4d9139a47641/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/00b00d76-1313-46cd-af96-4d9139a47641/session/6268416b-0db6-431e-ab87-d31e9a10ea13/turn "HTTP/1.1 200 OK"



👤 User Query:
I want you to create a new Docling document on Quantum Computing.
    To do this, you will create a document by invoking `create_new_docling_document` with Quantum Computing as prompt.
    Add a title and 3 paragraphs describing the topic, one by one, e.g. using the add_x_to_docling_document tools.
    At the end of the writing, you must save the document by calling `save_docling_document` and return me the filepath of
    the saved document in markdown format provided by the last tool.
    


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/00b00d76-1313-46cd-af96-4d9139a47641/session/6268416b-0db6-431e-ab87-d31e9a10ea13/turn/0c12d6df-314c-446d-ac68-8f1dd34475a8/resume "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/00b00d76-1313-46cd-af96-4d9139a47641/session/6268416b-0db6-431e-ab87-d31e9a10ea13/turn/0c12d6df-314c-446d-ac68-8f1dd34475a8/resume "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/00b00d76-1313-46cd-af96-4d9139a47641/session/6268416b-0db6-431e-ab87-d31e9a10ea13/turn/0c12d6df-314c-446d-ac68-8f1dd34475a8/resume "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/invoke "HTTP/1.1 200


---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{"thought": "I will create a new Docling document on Quantum Computing by using the create_new_docling_document tool with the prompt as the title of the document.", "action": {"tool_name": "create_new_docling_document", "tool_params": [{"name": "prompt", "value": "Quantum Computing"}]}, "answer": null}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='{\n  "document_key": "15299acfdad14d21b8a26de24e7c58b4",\n  "prompt": "Quantum Computing"\n}',
│   │   type='text'
│   )
]

...


---------- 📍 Step 9: InferenceStep ----------
🤖 Model Response:
{"thought": "I will add the third paragraph to the document.", "action": {"tool_name": "add_paragraph_to_docling_document", "tool_params": [{"name": "document_key", "value": "15299acfdad14d21b8a26de24e7c58b4"}, {"name": "paragraph", "value": "Quantum Computing has the potential to revolutionize many fields, including chemistry and materials science. It can be used to simulate complex systems and optimize processes, leading to breakthroughs in fields such as drug discovery and materials engineering."}]}, "answer": null}


---------- 📍 Step 10: ToolExecutionStep ----------
🔧 Executing tool...


[TextContentItem(text='{\n  "document_key": "15299acfdad14d21b8a26de24e7c58b4"\n}', type='text')]


---------- 📍 Step 11: InferenceStep ----------
🤖 Model Response:
{
  "thought": "I will save the document by calling `save_docling_document` and return me the filepath of the saved document in markdown format provided by the last tool.",
  "action": {
    "tool_name": "save_docling_document",
    "tool_params": [
      {"name": "document_key", "value": "15299acfdad14d21b8a26de24e7c58b4"}
    ]
  },
  "answer": null
}


---------- 📍 Step 12: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='{\n  "md_file": "/Users/dol/codes/docling-mcp/_cache/15299acfdad14d21b8a26de24e7c58b4.md",\n  "json_file": "/Users/dol/codes/docling-mcp/_cache/15299acfdad14d21b8a26de24e7c58b4.json"\n}',
│   │   type='text'
│   )
]


---------- 📍 Step 13: InferenceStep ----------
🤖 Model Response:
{"thought": "I will save the document by calling `save_docling_document` and return me the filepath of the saved document in markdown format provided by the last tool.", "action": {"tool_name": "save_docling_document", "tool_params": [{"name": "document_key", "value": "15299acfdad14d21b8a26de24e7c58b4"}]}, "answer": "/Users/dol/codes/docling-mcp/_cache/15299acfdad14d21b8a26de24e7c58b4.md"}

========== Query processing completed ========== 

